# Troubleshooting map extents for ArcGIS print server

In [ ]:
import arcgis
from arcgis.gis import GIS, Item
from arcgis.geometry import Polygon, Envelope
from arcgis.map.symbols import SimpleFillSymbolEsriSFS, SimpleFillSymbolStyle
from arcgis.map.symbols import SimpleLineSymbolEsriSLS
from arcgis import map as AGOMap
import json, requests, tempfile
from PIL import Image as PILImage
from PIL import ImageStat
from io import BytesIO


Authenticate with ArcGIS Online

In [ ]:
# Print the version of the arcgis module
print(f"Running ArcGIS API for Python version: {arcgis.__version__}")
agoNotebook = False
# Define the GIS
if agoNotebook == False:
    try:
        import keyring
        service_name = "system" # Use the default local credential store
        success = False # Set initial state

        # Ask for the username
        while success == False:
            username_for_keyring = input("Enter your ArcGIS Online username:") # If you are using VS Code, the text input dialog box appears at the top of the window
            # Get the credential object
            credential = keyring.get_credential(service_name, username_for_keyring)
            # Check if the username is in the credential store
            if credential is None:
                print(f"'{username_for_keyring}' is not in the local system's credential store. Try another username.")
            # Retrieve the password, login and set the GIS portal
            else:
                password_from_keyring = keyring.get_password("system", username_for_keyring)
                portal_url = 'https://www.arcgis.com'  
                gis = GIS(portal_url, username=username_for_keyring, password=password_from_keyring)
                success = True
                # Print a success message with username and user's organization role
                print(f"Successfully logged in as: {gis.properties.user.username} (role: {gis.properties.user.role} userType: {gis.properties.user.userLicenseTypeId})")
    except ImportError:
        print("The 'keyring' module is not installed. Please install it using 'pip install keyring'.")
        print("Before re-running this cell, open a command line window on your machine and run the command:")
        print("# python -m keyring set system <your_ago_username>")
        print("If using Windows Powershell, use:")
        print("# ./python -m keyring set system <your_ago_username>")
        print("You will be prompted to enter your password")
        print("When you hit Enter/Return the password will be saved to your local credential store.")
else:
    gis = GIS("home")

In [ ]:
from pyproj import Transformer

def extent_webmercator_to_wgs84(extent):
    """
    Transforms an extent dict from Web Mercator (EPSG:3857/102100) to WGS84 (EPSG:4326).
    Returns a dict with xmin, ymin, xmax, ymax in lon/lat.
    """
    # Support both 102100 and 3857
    wkid = extent.get('spatialReference', {}).get('wkid', 102100)
    if wkid not in [102100, 3857]:
        raise ValueError(f"Unsupported spatial reference: {wkid}")

    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
    xmin, ymin = transformer.transform(extent['xmin'], extent['ymin'])
    xmax, ymax = transformer.transform(extent['xmax'], extent['ymax'])
    return {
        "xmin": xmin,
        "ymin": ymin,
        "xmax": xmax,
        "ymax": ymax,
        "spatialReference": {"wkid": 4326}
    }

def extent_wgs84_to_webmercator(extent):
    """
    Transforms an extent dict from WGS84 (EPSG:4326) to Web Mercator (EPSG:3857/102100).
    Returns a dict with xmin, ymin, xmax, ymax in Web Mercator.
    """
    wkid = extent.get('spatialReference', {}).get('wkid', 4326)
    if wkid != 4326:
        raise ValueError(f"Unsupported spatial reference: {wkid}")

    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    xmin, ymin = transformer.transform(extent['xmin'], extent['ymin'])
    xmax, ymax = transformer.transform(extent['xmax'], extent['ymax'])
    return {
        "xmin": xmin,
        "ymin": ymin,
        "xmax": xmax,
        "ymax": ymax,
        "spatialReference": {"wkid": 102100}
    }

def is_blank_image(image_path, threshold=5):
    """
    Determines if an image is blank (mostly white or black) based on a threshold.
    Converts the image to grayscale and checks the standard deviation of pixel values.
    A low standard deviation indicates a uniform image.
    """
    img = PILImage.open(image_path).convert('L')
    pixels = list(img.getdata())
    unique_values = set(pixels)
    # If only 1 or 2 unique values (e.g., all black, all white, or half black/half white), treat as blank
    if len(unique_values) <= 2:
        return True
    stat = ImageStat.Stat(img)
    return stat.stddev[0] < threshold  # fallback for nearly-uniform images

In [ ]:
test_extent =    {
    "extent": {'xmin': -20721638.00383418, 'ymin': -4782120.616062363, 'xmax': -20674208.827783294, 'ymax': -4744437.161117821, 'spatialReference': {'wkid': 102100}}

}
transformed = extent_webmercator_to_wgs84(test_extent['extent'])
print(json.dumps(transformed, indent=2))

In [ ]:
inverse_transform = extent_wgs84_to_webmercator(transformed)
print(json.dumps(inverse_transform, indent=2))

In [ ]:


# Create outline
outline = SimpleLineSymbolEsriSLS(color=[255, 0, 0], width=1)

# Create fill symbol
symbol = SimpleFillSymbolEsriSFS(
    style=SimpleFillSymbolStyle.esri_sfs_solid,
    color=[50, 100, 200, 0],
    outline=outline
)

# 1. Create the bounding box polygon geometry
# extent = entries[4]['media']['webmap']['extent']
extent = test_extent['extent']
rings = [[
    [extent['xmin'], extent['ymin']],
    [extent['xmin'], extent['ymax']],
    [extent['xmax'], extent['ymax']],
    [extent['xmax'], extent['ymin']],
    [extent['xmin'], extent['ymin']]
]]
polygon = {
    "rings": rings,
    "spatialReference": extent['spatialReference']
}


map3 = AGOMap(gis=gis, item=None)
map3.basemap.basemap = 'imagery'
map3.extent = extent
# Draw the bounding box as a polygon graphic
map3.content.draw(shape=polygon, popup=None,     
        symbol=symbol, attributes=None, title="Extent")

map3

In [ ]:
test_json = {
  "baseMap": {
    "baseMapLayers": [
      {
        "id": "World_Imagery_2017",
        "layerType": "ArcGISTiledMapServiceLayer",
        "opacity": 1,
        "visibility": True,
        "url": "https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer"
      }
    ],
    "title": "Imagery"
  },
  "operationalLayers": [],
  "spatialReference": {
    "wkid": 102100
  },
  "mapOptions": {
    "extent": {'xmin': -20721638.00383418, 'ymin': -4782120.616062363, 'xmax': -20674208.827783294, 'ymax': -4744437.161117821, 'spatialReference': {'wkid': 102100}}
  },
  "exportOptions": {
    "outputSize": [
      800,
      600
    ],
    "dpi": 96
  }
}

In [ ]:
test_url = "https://utility.arcgisonline.com/arcgis/rest/services/Utilities/PrintingTools/GPServer/Export%20Web%20Map%20Task/execute"
test_webmap = test_json
test_params = {
        "f": "json",
        "Web_Map_as_JSON": json.dumps(test_webmap),
        "Format": "PNG32",
        "Layout_Template": "MAP_ONLY"
        }

response = requests.post(test_url, data=test_params)
test_result = response.json()

if 'results' in test_result:
    image_url = test_result['results'][0]['value']['url']
    img_response = requests.get(image_url)
    if img_response.status_code == 200:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        temp_file.write(img_response.content)
        temp_file.close()
        img = PILImage.open(temp_file.name)
        print(temp_file.name)
        is_blank = is_blank_image(temp_file.name)
        if is_blank:
            print("Generated thumbnail is blank.")